<a href="https://colab.research.google.com/github/cookieukw/IA-Simples/blob/main/IA_Simples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### importando as dependências

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


### Montar o google drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Exemplo de dados de treinamento (substitua com seus próprios dados)

In [4]:
questions = ["Qual é o seu nome?", "Como você está?", "O que você gosta de fazer?"]
answers = ["Meu nome é ChatGPT.", "Estou bem, obrigado!", "Eu gosto de ajudar as pessoas."]

### Tokenização dos dados


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)
question_seqs = tokenizer.texts_to_sequences(questions)
answer_seqs = tokenizer.texts_to_sequences(answers)

### Padding das sequências para ter o mesmo comprimento

In [6]:
max_seq_length = max(len(seq) for seq in question_seqs + answer_seqs)
question_seqs = pad_sequences(question_seqs, maxlen=max_seq_length, padding='post')
answer_seqs = pad_sequences(answer_seqs, maxlen=max_seq_length, padding='post')


### Conversão das respostas em formato one-hot

In [7]:
vocab_size = len(tokenizer.word_index) + 1
answer_one_hot = []
for seq in answer_seqs:
    one_hot_seq = tf.keras.utils.to_categorical(seq, num_classes=vocab_size)
    answer_one_hot.append(one_hot_seq)

answer_one_hot = np.array(answer_one_hot)


### Construção do modelo

In [8]:
model = keras.Sequential([
    Embedding(input_dim=vocab_size, output_dim=200, input_length=max_seq_length),
    LSTM(200, return_sequences=True),  # Use return_sequences=True para saída em cada etapa de tempo
    Dense(vocab_size, activation='softmax')
])

### Compilação do modelo

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Treinamento do modelo

In [13]:
model.fit(question_seqs, answer_one_hot, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 3.1319 - accuracy: 0.0556
Epoch 2/1000
1/1 [==============================] - 0s 25ms/step - loss: 3.1124 - accuracy: 0.4444
Epoch 3/1000
1/1 [==============================] - 0s 22ms/step - loss: 3.0925 - accuracy: 0.4444
Epoch 4/1000
1/1 [==============================] - 0s 25ms/step - loss: 3.0714 - accuracy: 0.4444
Epoch 5/1000
1/1 [==============================] - 0s 25ms/step - loss: 3.0486 - accuracy: 0.4444
Epoch 6/1000
1/1 [==============================] - 0s 24ms/step - loss: 3.0231 - accuracy: 0.4444
Epoch 7/1000
1/1 [==============================] - 0s 23ms/step - loss: 2.9942 - accuracy: 0.4444
Epoch 8/1000
1/1 [==============================] - 0s 27ms/step - loss: 2.9608 - accuracy: 0.3889
Epoch 9/1000
1/1 [==============================] - 0s 23ms/step - loss: 2.9215 - accuracy: 0.3333
Epoch 10/1000
1/1 [==============================] - 0s 31ms/step - loss: 2.8749 - accuracy: 0.3333
Epoch 11/10

### Salvar modelo no google drive

In [14]:
# Substitua 'meu_modelo.keras' pelo nome de arquivo desejado
model.save('/content/drive/My Drive/meu_modelo.keras')


 ### Agora você pode usar o modelo treinado para responder a perguntas

In [15]:
input_question = "Qual é o seu nome?"
input_seq = tokenizer.texts_to_sequences([input_question])
input_seq = pad_sequences(input_seq, maxlen=max_seq_length, padding='post')
output_seq = model.predict(input_seq)

1/1 [==============================] - 1s 701ms/step


### Converter as sequências de saída em palavras

In [16]:
output_words = []
for seq in output_seq[0]:
    word = tokenizer.index_word.get(np.argmax(seq))
    if word:
        output_words.append(word)

### Juntar as palavras em uma frase de resposta

In [17]:
output_answer = " ".join(output_words)
print("Resposta:", output_answer)

Resposta: meu nome é chatgpt


### Tamanho da Camada LSTM:

```python
LSTM(128, return_sequences=True)```
Você pode ajustar o valor 128 para aumentar ou diminuir o tamanho da camada LSTM.

### Tamanho do Embedding:

```python
Embedding(input_dim=vocab_size, output_dim=128, input_length=max_seq_length) ```
O valor output_dim define a dimensionalidade do espaço de embedding. Você pode ajustá-lo para um valor maior ou menor.

### Número de Épocas de Treinamento:

```python
model.fit(question_seqs, answer_one_hot, epochs=100)
Você pode ajustar o número de épocas de treinamento alterando epochs=100 para um valor maior ou menor, dependendo do desempenho desejado.

### Taxa de Aprendizado (Otimizador):
A taxa de aprendizado pode ser configurada ao compilar o modelo com um otimizador, como o Adam. Por exemplo:

```python
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
Você pode especificar a taxa de aprendizado no otimizador, se necessário.

Função de Ativação:
A função de ativação na camada de saída pode ser alterada, dependendo do tipo de tarefa. Atualmente, está configurada como softmax:

```python
Dense(vocab_size, activation='softmax')```
Você pode usar outras funções de ativação, como sigmoid, relu, ou qualquer outra adequada ao seu problema.

Lembre-se de que fazer ajustes em um modelo de aprendizado profundo pode exigir experimentação para encontrar a combinação ideal de hiperparâmetros que funcione para o seu caso específico. Os valores sugeridos acima são apenas pontos de partida, e você pode ajustá-los com base em seus resultados de treinamento e validação.